In [1]:
import io
import time
import requests
import datetime
import pandas as pd

from util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'
path_data = './data'
path_csv = path_data + '/csv'
file_name_out_r = path_csv + '/covid19-' + level + '.csv'
file_name_pop_r = path_csv + '/popolazione-regioni.csv'
file_name_icu_r = path_csv + '/terapie-intensive.csv'

In [2]:
# Define today, yesterday and day before yesterday dates
date_tday = datetime.date.today()
date_yday = Functions.get_previous_date(date_tday)
date_bf_yday = Functions.get_previous_date(date_yday)
date_tday, date_yday, date_bf_yday

(datetime.date(2020, 10, 31),
 datetime.date(2020, 10, 30),
 datetime.date(2020, 10, 29))

In [3]:
# Today url for regions file
url_tday = Functions.get_url_date(level, date_tday)
# Yesterday url for regions file
url_yday = Functions.get_url_date(level, date_yday)
# Day before yesterday url for regions file
url_bf_yday = Functions.get_url_date(level, date_bf_yday)

url_tday, url_yday, url_bf_yday

('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201031.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201030.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201029.csv')

In [4]:
wait = True
if wait:
    while True:
        response = requests.get(url_tday)
        if response.status_code == 200:
            print('Presenti dati odierni')
            date_last = date_tday
            csv = response.content
            df_raw_r_1 = pd.read_csv(io.StringIO(csv.decode('utf-8')))
            df_raw_r_0 = pd.read_csv(url_yday)
            break
        else:
            print('Status code: {}. Retrying...'.format(status_code))
        time.sleep(20)
else:
    print('Dati di ieri')
    date_last = date_yday
    df_raw_r_1 = pd.read_csv(url_yday)
    df_raw_r_0 = pd.read_csv(url_bf_yday)

Presenti dati odierni


In [5]:
# State S
df_raw_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-31T17:00:00,ITA,1,Piemonte,45.073274,7.680687,2683,174,2857,29967,...,2436,2887,33429,4383,33634,37002,70636,1029059,636308,NaN
1,2020-10-31T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,146,7,153,1660,...,96,104,1259,168,2864,376,3240,41184,25954,NaN
2,2020-10-31T17:00:00,ITA,3,Lombardia,45.466794,9.190347,4033,392,4425,77586,...,6782,8919,96198,17535,158695,37049,195744,2944616,1834360,NaN
3,2020-10-31T17:00:00,ITA,5,Veneto,45.434905,12.338452,801,123,924,28255,...,2632,2697,25373,2401,24864,32089,56953,2328514,904337,NaN
4,2020-10-31T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,167,36,203,4929,...,597,726,5312,397,9630,1211,10841,539077,229633,NaN
5,2020-10-31T17:00:00,ITA,7,Liguria,44.411493,8.932699,1036,57,1093,7338,...,109,1068,18403,1774,20350,8258,28608,443676,229381,NaN
6,2020-10-31T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1223,125,1348,21865,...,1792,2046,27997,4631,41394,14447,55841,1589691,849335,In seguito a verifica sui dati sono stati elim...
7,2020-10-31T17:00:00,ITA,9,Toscana,43.769231,11.255889,1046,163,1209,26635,...,1424,2540,15074,1345,35553,8710,44263,1090874,727110,NaN
8,2020-10-31T17:00:00,ITA,10,Umbria,43.106758,12.388247,277,43,320,6234,...,388,483,3499,126,3079,7100,10179,298628,173851,NaN
9,2020-10-31T17:00:00,ITA,11,Marche,43.616760,13.518875,275,47,322,5783,...,464,502,6997,1019,14005,116,14121,320766,187640,NaN


In [6]:
# State S - 1
df_raw_r_0.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-30T17:00:00,ITA,1,Piemonte,45.073274,7.680687,2547,159,2706,27682,...,2497,2719,33006,4355,32595,35154,67749,1013484,623529,NaN
1,2020-10-30T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,127,7,134,1583,...,133,155,1253,166,2768,368,3136,40780,25710,NaN
2,2020-10-30T17:00:00,ITA,3,Lombardia,45.466794,9.190347,3698,370,4068,71161,...,7345,8960,94134,17462,149776,37049,186825,2897835,1807884,NaN
3,2020-10-30T17:00:00,ITA,5,Veneto,45.434905,12.338452,777,117,894,25653,...,2921,3012,25321,2388,24718,29538,54256,2310397,898237,NaN
4,2020-10-30T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,161,35,196,4339,...,427,505,5186,394,9019,1096,10115,532815,227470,NaN
5,2020-10-30T17:00:00,ITA,7,Liguria,44.411493,8.932699,948,52,1000,7322,...,386,999,17469,1749,19564,7976,27540,437287,226812,NaN
6,2020-10-30T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1157,119,1276,20145,...,1708,1763,27764,4612,40312,13485,53797,1570748,842483,In seguito a verifica sui dati comunicati nei ...
7,2020-10-30T17:00:00,ITA,9,Toscana,43.769231,11.255889,1007,153,1160,25260,...,2450,2765,13983,1320,33468,8255,41723,1074698,717521,effettuati 12488 tamponi rapidi antigenici di ...
8,2020-10-30T17:00:00,ITA,10,Umbria,43.106758,12.388247,260,42,302,5864,...,595,729,3409,121,2996,6700,9696,294451,171539,NaN
9,2020-10-30T17:00:00,ITA,11,Marche,43.616760,13.518875,262,39,301,5340,...,505,524,6965,1013,13554,65,13619,317571,185912,NaN


In [7]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-31T17:00:00,ITA,1,Piemonte,45.073274,7.680687,2683,174,2857,29967,...,2436,2887,33429,4383,33634,37002,70636,1029059,636308,NaN
1,2020-10-31T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,146,7,153,1660,...,96,104,1259,168,2864,376,3240,41184,25954,NaN
2,2020-10-31T17:00:00,ITA,3,Lombardia,45.466794,9.190347,4033,392,4425,77586,...,6782,8919,96198,17535,158695,37049,195744,2944616,1834360,NaN
3,2020-10-31T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,358,27,385,6640,...,775,937,9745,750,13562,3958,17520,522486,234484,NaN
4,2020-10-31T17:00:00,ITA,5,Veneto,45.434905,12.338452,801,123,924,28255,...,2632,2697,25373,2401,24864,32089,56953,2328514,904337,NaN
5,2020-10-31T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,167,36,203,4929,...,597,726,5312,397,9630,1211,10841,539077,229633,NaN
6,2020-10-31T17:00:00,ITA,7,Liguria,44.411493,8.932699,1036,57,1093,7338,...,109,1068,18403,1774,20350,8258,28608,443676,229381,NaN
7,2020-10-31T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1223,125,1348,21865,...,1792,2046,27997,4631,41394,14447,55841,1589691,849335,In seguito a verifica sui dati sono stati elim...
8,2020-10-31T17:00:00,ITA,9,Toscana,43.769231,11.255889,1046,163,1209,26635,...,1424,2540,15074,1345,35553,8710,44263,1090874,727110,NaN
9,2020-10-31T17:00:00,ITA,10,Umbria,43.106758,12.388247,277,43,320,6234,...,388,483,3499,126,3079,7100,10179,298628,173851,NaN


In [8]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,11,2020-10-30T17:00:00,ITA,1,Piemonte,45.073274,7.680687,2547,159,2706,...,2497,2719,33006,4355,32595,35154,67749,1013484,623529,NaN
1,17,2020-10-30T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,127,7,134,...,133,155,1253,166,2768,368,3136,40780,25710,NaN
2,8,2020-10-30T17:00:00,ITA,3,Lombardia,45.466794,9.190347,3698,370,4068,...,7345,8960,94134,17462,149776,37049,186825,2897835,1807884,NaN
3,19,2020-10-30T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,351,27,378,...,412,572,9588,745,12791,3792,16583,516331,232239,NaN
4,18,2020-10-30T17:00:00,ITA,5,Veneto,45.434905,12.338452,777,117,894,...,2921,3012,25321,2388,24718,29538,54256,2310397,898237,NaN
5,5,2020-10-30T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,161,35,196,...,427,505,5186,394,9019,1096,10115,532815,227470,NaN
6,7,2020-10-30T17:00:00,ITA,7,Liguria,44.411493,8.932699,948,52,1000,...,386,999,17469,1749,19564,7976,27540,437287,226812,NaN
7,4,2020-10-30T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1157,119,1276,...,1708,1763,27764,4612,40312,13485,53797,1570748,842483,In seguito a verifica sui dati comunicati nei ...
8,15,2020-10-30T17:00:00,ITA,9,Toscana,43.769231,11.255889,1007,153,1160,...,2450,2765,13983,1320,33468,8255,41723,1074698,717521,effettuati 12488 tamponi rapidi antigenici di ...
9,16,2020-10-30T17:00:00,ITA,10,Umbria,43.106758,12.388247,260,42,302,...,595,729,3409,121,2996,6700,9696,294451,171539,NaN


In [9]:
# Print raw columns
# print(df_r_1.columns, df_r_0.columns)
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rf_1 = df_r_1.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0 = df_r_0.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
# df_rf_1.columns, df_rf_0.columns

In [10]:
df_rf_0 = df_rf_0.sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0;

In [11]:
df_r = df_rf_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(file_name_pop_r)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-10-31T17:00:00,1,Piemonte,2683,174,2857,29967,32824,2436,2887,33429,4383,33634,37002,70636,1029059,636308,4341375
1,2020-10-31T17:00:00,2,Valle d'Aosta,146,7,153,1660,1813,96,104,1259,168,2864,376,3240,41184,25954,125501
2,2020-10-31T17:00:00,3,Lombardia,4033,392,4425,77586,82011,6782,8919,96198,17535,158695,37049,195744,2944616,1834360,10103969
3,2020-10-31T17:00:00,4,Trentino-Alto Adige,358,27,385,6640,7025,775,937,9745,750,13562,3958,17520,522486,234484,1074819
4,2020-10-31T17:00:00,5,Veneto,801,123,924,28255,29179,2632,2697,25373,2401,24864,32089,56953,2328514,904337,4907704
5,2020-10-31T17:00:00,6,Friuli Venezia Giulia,167,36,203,4929,5132,597,726,5312,397,9630,1211,10841,539077,229633,1211357
6,2020-10-31T17:00:00,7,Liguria,1036,57,1093,7338,8431,109,1068,18403,1774,20350,8258,28608,443676,229381,1543127
7,2020-10-31T17:00:00,8,Emilia-Romagna,1223,125,1348,21865,23213,1792,2046,27997,4631,41394,14447,55841,1589691,849335,4467118
8,2020-10-31T17:00:00,9,Toscana,1046,163,1209,26635,27844,1424,2540,15074,1345,35553,8710,44263,1090874,727110,3722729
9,2020-10-31T17:00:00,10,Umbria,277,43,320,6234,6554,388,483,3499,126,3079,7100,10179,298628,173851,880285


In [12]:
# Merge with icu data
df_icu_r = pd.read_csv(file_name_icu_r)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-10-31T17:00:00,1,Piemonte,2683,174,2857,29967,32824,2436,2887,33429,4383,33634,37002,70636,1029059,636308,4341375,367
1,2020-10-31T17:00:00,2,Valle d'Aosta,146,7,153,1660,1813,96,104,1259,168,2864,376,3240,41184,25954,125501,20
2,2020-10-31T17:00:00,3,Lombardia,4033,392,4425,77586,82011,6782,8919,96198,17535,158695,37049,195744,2944616,1834360,10103969,983
3,2020-10-31T17:00:00,4,Trentino-Alto Adige,358,27,385,6640,7025,775,937,9745,750,13562,3958,17520,522486,234484,1074819,106
4,2020-10-31T17:00:00,5,Veneto,801,123,924,28255,29179,2632,2697,25373,2401,24864,32089,56953,2328514,904337,4907704,825
5,2020-10-31T17:00:00,6,Friuli Venezia Giulia,167,36,203,4929,5132,597,726,5312,397,9630,1211,10841,539077,229633,1211357,175
6,2020-10-31T17:00:00,7,Liguria,1036,57,1093,7338,8431,109,1068,18403,1774,20350,8258,28608,443676,229381,1543127,209
7,2020-10-31T17:00:00,8,Emilia-Romagna,1223,125,1348,21865,23213,1792,2046,27997,4631,41394,14447,55841,1589691,849335,4467118,516
8,2020-10-31T17:00:00,9,Toscana,1046,163,1209,26635,27844,1424,2540,15074,1345,35553,8710,44263,1090874,727110,3722729,415
9,2020-10-31T17:00:00,10,Umbria,277,43,320,6234,6554,388,483,3499,126,3079,7100,10179,298628,173851,880285,70


In [13]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [14]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_rf_1['ricoverati_con_sintomi'] - df_rf_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_rf_1['terapia_intensiva'] - df_rf_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_rf_1['totale_ospedalizzati'] - df_rf_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_rf_1['isolamento_domiciliare'] - df_rf_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_rf_1['nuovi_positivi'] - df_rf_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_rf_1['tamponi'] - df_rf_0['tamponi']
df_r['nuovi_casi_testati'] = df_rf_1['casi_testati'] - df_rf_0['casi_testati']
df_r['nuovi_deceduti'] = df_rf_1['deceduti'] - df_rf_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_rf_1['dimessi_guariti'] - df_rf_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-10-31T17:00:00,3,Lombardia,4033,392,4425,77586,82011,6782,8919,...,39.88,6425,-41,46781,26476,73,2064,19.07,0.08827,1.94
1,2020-10-31T17:00:00,15,Campania,1403,168,1571,42159,43730,3380,3669,...,39.34,3355,483,20860,16746,14,275,17.59,0.06341,0.96
2,2020-10-31T17:00:00,1,Piemonte,2683,174,2857,29967,32824,2436,2887,...,47.41,2285,168,15575,12779,28,423,18.54,0.06650,1.63
3,2020-10-31T17:00:00,5,Veneto,801,123,924,28255,29179,2632,2697,...,14.91,2602,-315,18117,6100,13,52,14.89,0.05495,1.16
4,2020-10-31T17:00:00,9,Toscana,1046,163,1209,26635,27844,1424,2540,...,39.28,1375,-225,16176,9589,25,1091,15.70,0.06823,1.19
5,2020-10-31T17:00:00,12,Lazio,1944,182,2126,31780,33906,2159,2289,...,24.36,2081,43,25767,22051,22,108,8.88,0.03902,0.79
6,2020-10-31T17:00:00,8,Emilia-Romagna,1223,125,1348,21865,23213,1792,2046,...,24.22,1720,283,18943,6852,19,233,10.80,0.04580,1.25
7,2020-10-31T17:00:00,7,Liguria,1036,57,1093,7338,8431,109,1068,...,27.27,16,69,6389,2569,25,934,16.72,0.06921,1.85
8,2020-10-31T17:00:00,19,Sicilia,962,122,1084,13358,14442,878,952,...,22.68,806,-32,8106,5303,18,56,11.74,0.01916,0.44
9,2020-10-31T17:00:00,4,Trentino-Alto Adige,358,27,385,6640,7025,775,937,...,25.47,768,365,6155,2245,5,157,15.22,0.08718,1.63


In [15]:
# Write pre-processed data
df_r.to_csv(file_name_out_r, index=False)